# Rock Paper Scissors - Irrational Search Agent

This attempts a Password Attack against [IrrationalAgent](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-irrational-agent)

Its only vulnerability is Password Attack.
Pleased to meet you, won't you guess my name?

There are an uncountable infinity of irrational numbers,
but a rather limited number of useful irrationals with pronounceable names

This will only work if the opponent has chosen to play an irrational agent
but has chosen to play a popular named irrational
and has also chosen the same trinary encoding algorithm

If the opponent is not playing a known irrational sequence
then the true Nash Equilibrium is to play a secret and unnamed irrational sequence

IrrationalSearchAgent() either wins or it draws, but it never statistically loses.

# Irrational Agent

This is both the base class, and the target of the attack

For a more extended discussion see the [IrrationalAgent](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-irrational-agent) notebook

In [ ]:
%%writefile IrrationalSearchAgent.py
# Source: https://www.kaggle.com/jamesmcguigan/random-seed-search-irrational-agent/
# Source: https://github.com/JamesMcGuigan/ai-games/blob/master/games/rock-paper-scissors/rng/IrrationalAgent.py

import re
import time
from typing import List, Union

from mpmath import mp
mp.dps = 2048  # more than 1000 as 10%+ of chars will be dropped


def encode_irrational(irrational: Union[str,mp.mpf], offset=0) -> List[int]:
    """
    Encode the irrational number into trinary
    The irrational is converted to a string, "0"s are removed
    Then each of the digits is converted to an integer % 3 and added the to sequence
    """
    if isinstance(irrational, list) and all([ 0 <= n <= 2 for n in irrational ]):
        return irrational  # prevent double encoding

    string   = re.sub('[^1-9]', '', str(irrational))
    sequence = [
        ( int(c) + int(offset) ) % 3
        for c in string
    ]
    assert len(sequence) >= 1000
    return sequence



class IrrationalAgent():
    """
    Play an fixed sequence of moves derived the digits of an irrational number

    Irrational numbers are more mathematically pure source of randomness than
    the repeating numbers used by the Mersenne Twister RNG

    This agent is pure Chaos and contains no Order capable of being exploited once the game has started

    Its only vulnerability is Password Attack.
    Pleased to meet you, won't you guess my name?

    There are an uncountable infinity of irrational numbers
    Your choice of irrational is your password
    Be irrational in your choice of irrational if want this agent to be secure
    Alternatively choose a popular irrational with an offset to attack a specific agent

    This is the true Nash Equilibrium solution to Rock Paper Scissors
    """

    irrationals = {
        name: encode_irrational(irrational)
        for name, irrational in {
            'pi':       mp.pi(),
            'phi':      mp.phi(),
            'e':        mp.e(),
            'sqrt2':    mp.sqrt(2),
            'sqrt5':    mp.sqrt(5),
            'euler':    mp.euler(),
            'catalan':  mp.catalan(),
            'apery':    mp.apery(),
            # 'khinchin':  mp.khinchin(),  # slow
            # 'glaisher':  mp.glaisher(),  # slow
            # 'mertens':   mp.mertens(),   # slow
            # 'twinprime': mp.twinprime(), # slow
        }.items()
    }


    def __init__(self, name='irrational', irrational: Union[str,mp.mpf] = None, offset=0, verbose=True):
        # Irrational numbers are pure random sequences that are immune to random seed search
        # Using name == 'irrational' causes the number to be reset each new game
        if irrational is not None and ( name == 'irrational' or name in self.irrationals.keys() ):
            name = 'secret'
        if name == 'irrational':
            irrational = self.generate_secure_irrational()
        if name in self.irrationals.keys():
            irrational = self.irrationals[name]
        self.irrational = self.encode_irrational(irrational, offset=offset)

        self.name       = name
        self.offset     = offset
        self.verbose    = verbose
        self.reset()


    def reset(self):
        """
        Reset on the first turn of every new game
        This allows a single instance to be run in a loop for testing
        """
        self.history = {
            "action":   [],
            "opponent": []
        }
        if self.name == 'irrational':
            irrational      = self.generate_secure_irrational()
            self.irrational = self.encode_irrational(irrational, offset=self.offset)



    def __call__(self, obs, conf):
        return self.agent(obs, conf)

    def agent(self, obs, conf):
        """ Wrapper function for setting state in child classes """

        # Generate a new history and irrational seed for each new game
        if obs.step == 0:
            self.reset()

        # Keep a record of opponent and action state
        if obs.step > 0:
            self.history['opponent'].append(obs.lastOpponentAction)

        # This is where the subclassable agent logic happens
        action = self.action(obs, conf)

        # Keep a record of opponent and action state
        self.history['action'].append(action)
        return action


    def action(self, obs, conf):
        """ Play the next digit in a fixed irrational sequence """
        action = int(self.irrational[ obs.step % len(self.irrational) ])
        action = (action + self.offset) % conf.signs
        if self.verbose:
            name = self.__class__.__name__ + ':' + self.name + (f'+{self.offset}' if self.offset else '')
            opponent = ( self.history['opponent'] or [None] )[-1]
            expected = ( action - 1 ) % 3
            print(f"{obs.step:4d} | {opponent}{self.win_symbol()} > action {action} | " +
                  f"{name}")
        return action


    @staticmethod
    def generate_secure_irrational():
        """
        Be irrational in your choice of irrational if want this agent to be secure
        """
        irrational = sum([
            mp.sqrt(n) * (time.monotonic_ns() % 2**32)
            for n in range(2, 5 + (time.monotonic_ns() % 1024))
        ])
        return irrational


    @classmethod
    def encode_irrational(cls, irrational: Union[str,mp.mpf], offset=0) -> List[int]:
        if irrational is None:
            irrational = cls.generate_secure_irrational()
        return encode_irrational(irrational, offset)



    ### Logging

    def win_symbol(self):
        """ Symbol representing the reward from the previous turn """
        action   = ( self.history['action']   or [None] )[-1]
        opponent = ( self.history['opponent'] or [None] )[-1]
        if isinstance(action, int) and isinstance(opponent, int):
            if action % 3 == (opponent + 1) % 3: return '+'  # win
            if action % 3 == (opponent + 0) % 3: return '|'  # draw
            if action % 3 == (opponent - 1) % 3: return '-'  # loss
        return ' '


irrational_instance = IrrationalAgent(name='pi', offset=0)
def irrational_agent(obs, conf):
    return irrational_instance.agent(obs, conf)


# Irrational Search Agent

This is the agent which will test if the opponent is playing an irrational number agent
but has chosen to use one of the more popular and named irrational numbers

In [ ]:
%%writefile -a IrrationalSearchAgent.py
# Source: https://www.kaggle.com/jamesmcguigan/random-seed-search-irrational-search-agent/
# Source: https://github.com/JamesMcGuigan/ai-games/blob/master/games/rock-paper-scissors/rng/IrrationalSearchAgent.py
from typing import List, Tuple, Union

from mpmath import mp

# from IrrationalAgent import IrrationalAgent


class IrrationalSearchAgent(IrrationalAgent):
    """
    This attempts a Password Attack against IrrationalAgent

    Its only vulnerability is Password Attack.
    Pleased to meet you, won't you guess my name?

    There are an uncountable infinity of irrational numbers,
    but a rather limited number of useful irrationals with pronounceable names

    This will only work if the opponent has chosen to play an irrational agent
    but has chosen to play a popular named irrational
    and has also chosen the same trinary encoding algorithm

    If the opponent is not playing a known irrational sequence
    then the true Nash Equilibrium is to play a secret and unnamed irrational sequence
    """

    def __init__(self, name='irrational', irrational=None, search: List[Union[str, mp.mpf]]=None, verbose=True):
        search = {
            f'irrational#{n}': self.encode_irrational(number)
            for n, number in enumerate(search)
        } if search is not None else {}
        self.irrationals = { **self.__class__.irrationals, **search }
        super().__init__(name=name, irrational=irrational, verbose=verbose)


    def action(self, obs, conf):
        expected, irrational_name = self.search_irrationals(self.history['opponent'])
        if expected is not None:
            action   = (expected + 1) % conf.signs
            opponent = ( self.history['opponent'] or [None] )[-1]
            if self.verbose:
                print(f"{obs.step:4d} | {opponent}{self.win_symbol()} > action {expected} -> {action} | " +
                      f"Found Irrational: {irrational_name}")
        else:
            action = super().action(obs, conf)  # play our own irrational number sequence
        return action


    def search_irrationals(self, sequence: List[int]) -> Tuple[int, str]:
        """
        Search through list of named irrational sequences
        if found, return the next expected number in the sequence along with the name
        """
        expected, irrational_name = None, None
        if len(sequence):
            for offset in [0,1,2]:
                for irrational_name, irrational in self.irrationals.items():
                    irrational = [ (n + offset) % 3 for n in irrational[:len(sequence)+1] ]
                    if irrational[:len(sequence)] == sequence:
                        expected = irrational[len(sequence)]
                        break
                else: continue
                break
        return expected, irrational_name




irrational_search_instance = IrrationalSearchAgent()
def irrational_search_agent(obs, conf):
    return irrational_search_instance.agent(obs, conf)


In [ ]:
%run IrrationalSearchAgent.py

# Unit Tests

In [ ]:
%%writefile test_IrrationalSeedSearch.py
import pytest
from kaggle_environments import evaluate

from IrrationalSearchAgent import IrrationalAgent
from IrrationalSearchAgent import IrrationalSearchAgent
import numpy as np

@pytest.mark.parametrize("name",   IrrationalSearchAgent.irrationals.keys())
@pytest.mark.parametrize("offset", [0,1,2])
def test_IrrationalSearchAgent_vs_named_irrational(name, offset):
    """ Assert we can find the full irrational sequence every time """
    episodeSteps = 1000
    results = evaluate(
        "rps",
        [
            IrrationalAgent(name=name, offset=offset),
            IrrationalSearchAgent()
        ],
        configuration={
            "episodeSteps": episodeSteps,
            # "actTimeout":   1000,
        },
    )
    assert (results[0][0] + episodeSteps/2.1) < results[0][1]



def test_IrrationalSearchAgent_vs_Irrational():
    episodeSteps = 100
    results = evaluate(
        "rps",
        [
            IrrationalAgent(),
            IrrationalSearchAgent()
        ],
        configuration={
            "episodeSteps": episodeSteps,
            # "actTimeout":   1000,
        },
        num_episodes=100,
        # debug=True, # pull request
    )
    results = np.array(results).reshape((-1,2))
    totals  = np.mean(results, axis=0)
    std     = np.std(results, axis=0).round(2)
    winrate = [ np.sum(results[:,0]-20 > results[:,1]),
                np.sum(results[:,0]+20 < results[:,1]) ]

    print('results', results)
    print('totals',  totals)
    print('std',     std)
    print('winrate', winrate)

    assert len(results[ results == None ]) == 0    # No errored matches
    assert np.abs(totals[0]) < 0.2 * episodeSteps  # totals are within 20%
    assert np.abs(totals[1]) < 0.2 * episodeSteps  # totals are within 20%
    assert np.abs(std[0])    < 0.2 * episodeSteps  # std  within 20%
    assert np.abs(std[1])    < 0.2 * episodeSteps  # std  within 20%


In [ ]:
!pytest -v test_IrrationalSeedSearch.py

# Demonstration

This is more advanced than the simple Anti-PI bot, which is only capable of attacking a single named opponent.

As can be seen here and above with the unit tests, the IrrationalSearchAgent() is capable of performing a Password Attack and getting a perfect score against a range of popular irrational numbers, as well as their counter and counter-counter bots.

The IrrationalSearchAgent() itself cannot be countered by its own technique. In the absense of a known winning strategy it opts for a draw, by playing an unguessable secret irrational number, which is the true Nash Equlibrium strategy. This agent either wins or it draws, but it never statistically loses.

In [ ]:
from kaggle_environments import make, evaluate

env = make("rps", configuration={"episodeSteps": 25}, debug=True)
env.run([ IrrationalAgent(name='pi'), IrrationalSearchAgent() ])
env.render(mode="ipython", width=400, height=400)

In [ ]:
from kaggle_environments import make, evaluate

env = make("rps", configuration={"episodeSteps": 25}, debug=True)
env.run([ IrrationalAgent(name='irrational'), IrrationalSearchAgent() ])
env.render(mode="ipython", width=400, height=400)

# Statistics

- Against a named irrational number, it always wins
- Against a secret irrational number, it is statistically a draw
- These statistics match up with the extended discussion from the [IrrationalAgent](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-irrational-agent) notebook

In [ ]:
%%time
import numpy as np

agents  = [ IrrationalSearchAgent(), IrrationalAgent(name='pi') ]
results = evaluate(
    "rps",
    agents,
    num_episodes=100,
)
results = np.array(results).reshape((-1,2))
results[ results == None ] = -1

print([ getattr(agent, '__name__', agent.__class__.__name__) for agent in agents ])
print('winrate', [ np.sum(results[:,0]-20 > results[:,1]),
                   np.sum(results[:,0]+20 < results[:,1])
                 ], '/', len(results))
print('totals ', np.sum(results, axis=0))
print('std    ', np.std(results, axis=0).round(2))

In [ ]:
%%time
import numpy as np

agents  = [ IrrationalSearchAgent(), IrrationalAgent() ]
results = evaluate(
    "rps",
    agents,
    num_episodes=100,
)
results = np.array(results).reshape((-1,2))
results[ results == None ] = -1

print([ getattr(agent, '__name__', agent.__class__.__name__) for agent in agents ])
print('winrate', [ np.sum(results[:,0]-20 > results[:,1]),
                   np.sum(results[:,0]+20 < results[:,1])
                 ], '/', len(results))
print('totals ', np.sum(results, axis=0))
print('std    ', np.std(results, axis=0).round(2))

# Further Reading

This notebook is part of a series exploring Rock Paper Scissors:

Irrational
- [PI Bot](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-pi-bot)
- [Anti-PI Bot](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-anti-pi-bot)
- [Anti-Anti-PI Bot](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-anti-anti-pi-bot)
- [Irrational Agent](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-irrational-agent)
- [Irrational Search Agent](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-irrational-search-agent)
- [Random Seed Search Nash Equlibrium Opening Book](https://www.kaggle.com/jamesmcguigan/random-seed-search-nash-equlibrium-opening-book)

RNG
- [Random Agent](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-random-agent)
- [RNG Statistics](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-rng-statistics)

Sequence
- [De Bruijn Sequence](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-de-bruijn-sequence)

Opponent Response
- [Anti-Rotn](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-anti-rotn)
- [Sequential Strategies](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-sequential-strategies)

Statistical 
- [Weighted Random Agent](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-weighted-random-agent)
- [Anti-Rotn Weighted Random](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-anti-rotn-weighted-random)
- [Statistical Prediction](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-statistical-prediction)

Memory Patterns
- [Naive Bayes](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-naive-bayes)
- [Memory Patterns](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-memory-patterns)

Decision Tree
- [XGBoost](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-xgboost)
- [Multi Stage Decision Tree](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-multi-stage-decision-tree)
- [Decision Tree Ensemble](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-decision-tree-ensemble)

Neural Networks
- [LSTM](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-lstm)

Ensemble
- [Multi Armed Stats Bandit](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-multi-armed-stats-bandit)

RoShamBo Competition Winners
- [Iocaine Powder](https://www.kaggle.com/jamesmcguigan/rps-roshambo-comp-iocaine-powder)
- [Greenberg](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-greenberg)